In [1]:
import pandas as pd
import datetime
import requests
import json
import sys, os
import itertools
import time
import numpy as np
from os import path
from pathlib import Path
from binance.spot import Spot
from twelvedata import TDClient
from sklearn import metrics
from sklearn.inspection import permutation_importance
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import itertools
import xgboost as xgb
import seaborn as sns
from typing import Union, Dict, List
import talib
import ta
import shap
sys.path.insert(0, os.path.join(os.getcwd(), ".."))
import fin_utilities 
cfg = fin_utilities.__cfg_reading("pred")
import my_functions
from coinmetrics.api_client import CoinMetricsClient
import matplotlib as mpl
# Set the default color cycle
mpl.rcParams.update(mpl.rcParamsDefault)


#td = TDClient(apikey=cfg['TWELVEDATA']['API'])  

PROJECT_DIR = eval(cfg['PROJECT_PATH'])
DATA_PATH   = PROJECT_DIR / cfg['DATA_FOLDER']
SOURCE      = cfg['API_DATA_SOURCE']
URL         = cfg[SOURCE]['API_URL_HIST_DATA']
COLUMNS     = cfg[SOURCE]['COLUMN_NAMES']
PARAMS      = cfg[SOURCE]['REQ_PARAMS']
PAIR        = cfg['SYMBOL'] + cfg['STABLECOIN']
MAX_LENGTH  = cfg['MAX_TRADE_DURATION']
XGB_PARAM   = cfg['xgb']


c:\Users\ma.orlandi\Anaconda3\envs\obb\lib\site-packages\torch\onnx\_internal\_beartype.py:30: UserWarning: module 'beartype.roar' has no attribute 'BeartypeDecorHintPep585DeprecationWarning'
  warnings.warn(f"{e}")


In [2]:
client = CoinMetricsClient()
print(client)

In [5]:
frequency  = "1h"
start_time = "2009-01-01"
end_time   = "2023-02-05"
asset      = ["btc"]
metrics_a = [
    'AdrActCnt',
]

df_mcap = client.get_asset_metrics(
                                    assets    = asset,
                                    metrics   = metrics_a,
                                    frequency = frequency,
                                    start_time= start_time,
                                    end_time  = end_time,
                                    page_size = 10000
                                ).to_dataframe()
df_mcap

HTTPError: 403 Client Error: Forbidden for url: https://community-api.coinmetrics.io/v4/timeseries/asset-metrics?&assets=btc&metrics=AdrActCnt&frequency=1h&page_size=10000&paging_from=start&start_time=2009-01-01&end_time=2023-02-05

In [4]:
frequency  = "1d"
start_time = "2009-10-01"
end_time   = "2023-02-05"
asset      = ["btc"]
metrics_a = [
    'CapAct1yrUSD',
    'CapMVRVCur',
    'CapMVRVFF',
    'CapMrktCurUSD',
    'CapMrktEstUSD',
    'CapMrktFFUSD',
    'CapRealUSD',
    'AdrActCnt',
    'FeeTotUSD',
    'FeeMeanUSD',
    'DiffLast',
    'DiffMean',
    'FlowInExNtv',
    'FlowInExUSD',
    'FlowOutExNtv',
    'FlowOutExUSD',
    'HashRate',		
    'HashRate30d',
    'NDF',
    'FlowTfrFromExCnt'

]

df_mcap = client.get_asset_metrics(
                                    assets    = asset,
                                    metrics   = metrics_a,
                                    frequency = frequency,
                                    start_time= start_time,
                                    end_time  = end_time,
                                    page_size = 10000
                                ).to_dataframe()
df_mcap

,asset,time,AdrActCnt,CapAct1yrUSD,CapMVRVCur,CapMVRVFF,CapMrktCurUSD,CapMrktEstUSD,CapMrktFFUSD,CapRealUSD,...,FlowOutExNtv-status-time,FlowOutExUSD,FlowOutExUSD-status,FlowOutExUSD-status-time,FlowTfrFromExCnt,FlowTfrFromExCnt-status,FlowTfrFromExCnt-status-time,HashRate,HashRate30d,NDF
0,btc,2009-10-01 00:00:00+00:00,82,None,None,None,None,None,None,0.0,...,2020-02-28T09:17:32.260416000Z,None,None,None,0,reviewed,2020-02-28T09:17:32.260416000Z,0.000004,0.000004,0.061605
1,btc,2009-10-02 00:00:00+00:00,83,None,None,None,None,None,None,0.0,...,2020-02-28T09:17:32.260416000Z,None,None,None,0,reviewed,2020-02-28T09:17:32.260416000Z,0.000004,0.000004,0.061835
2,btc,2009-10-03 00:00:00+00:00,58,None,None,None,None,None,None,0.0,...,2020-02-28T09:17:32.260416000Z,None,None,None,0,reviewed,2020-02-28T09:17:32.260416000Z,0.000003,0.000004,0.061687
3,btc,2009-10-04 00:00:00+00:00,53,None,None,None,None,None,None,0.0,...,2020-02-28T09:17:32.260416000Z,None,None,None,0,reviewed,2020-02-28T09:17:32.260416000Z,0.000003,0.000004,0.061553
4,btc,2009-10-05 00:00:00+00:00,55,None,None,None,None,None,None,0.0,...,2020-02-28T09:17:32.260416000Z,None,None,None,0,reviewed,2020-02-28T09:17:32.260416000Z,0.000003,0.000004,0.061415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4871,btc,2023-02-01 00:00:00+00:00,1050528,154362621309.09605668301901291,1.196241930423,0.861856310148,457433839815.109798416131050605,457416998038.184547636,329567315183.87374300259239134,382392414261.365601,...,2023-02-02T01:25:30.937895000Z,411328777.57064553725068836,flash,2023-02-02T05:12:47.852427000Z,29864,flash,2023-02-02T05:08:21.563026000Z,267991996.867647,273309714.345232,0.324221
4872,btc,2023-02-02 00:00:00+00:00,983946,150681969171.599475637387303482,1.184795889398,0.853464431963,453182984739.087481763485849248,453165872083.4987139552,326449105796.707377751492843464,382498782106.23114,...,2023-02-03T01:11:56.528979000Z,594855629.048353884159735864,flash,2023-02-03T04:50:21.275206000Z,31856,flash,2023-02-03T04:45:28.013448000Z,283641162.984755,274045496.533696,0.324876
4873,btc,2023-02-03 00:00:00+00:00,988867,150273538776.28805361363899712,1.181415746292,0.850970452062,452049702984.097911637682895936,452034246444.1374432096,325610134544.146004676584546904,382633890231.091309,...,2023-02-04T02:29:51.085867000Z,486143535.38560819750909188,flash,2023-02-04T05:23:37.139284000Z,29804,flash,2023-02-04T05:30:10.766507000Z,260167412.400668,274639194.1772,0.324843
4874,btc,2023-02-04 00:00:00+00:00,961115,149647997797.290925613539563122,1.176437733184,0.847222179515,450328730991.77458087072933871,450312026811.61260685,324308272514.133222982192813364,382790111443.578796,...,2023-02-05T01:15:04.364549000Z,262106584.054421414531276526,flash,2023-02-05T04:17:54.679010000Z,21809,flash,2023-02-05T05:04:20.591605000Z,266035849.342477,274637599.469207,0.324793


In [15]:
#droppo tutte le colonne "status" inutili
df_mcap.drop(columns=[col for col in df_mcap.columns if 'status' in col], inplace=True)
df_mcap

,asset,time,AdrActCnt,CapAct1yrUSD,CapMVRVCur,CapMVRVFF,CapMrktCurUSD,CapMrktEstUSD,CapMrktFFUSD,CapRealUSD,...,FeeMeanUSD,FeeTotUSD,FlowInExNtv,FlowInExUSD,FlowOutExNtv,FlowOutExUSD,FlowTfrFromExCnt,HashRate,HashRate30d,NDF
0,btc,2009-01-03 00:00:00+00:00,0,None,None,None,None,None,None,0.0,...,None,None,0.0,None,0.0,None,0,None,None,None
1,btc,2009-01-04 00:00:00+00:00,0,None,None,None,None,None,None,0.0,...,None,None,0.0,None,0.0,None,0,None,None,None
2,btc,2009-01-05 00:00:00+00:00,0,None,None,None,None,None,None,0.0,...,None,None,0.0,None,0.0,None,0,None,None,None
3,btc,2009-01-06 00:00:00+00:00,0,None,None,None,None,None,None,0.0,...,None,None,0.0,None,0.0,None,0,None,None,None
4,btc,2009-01-07 00:00:00+00:00,0,None,None,None,None,None,None,0.0,...,None,None,0.0,None,0.0,None,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5142,btc,2023-02-01 00:00:00+00:00,1050528,154362621309.09605668301901291,1.196241930423,0.861856310148,457433839815.109798416131050605,457416998038.184547636,329567315183.87374300259239134,382392414261.365601,...,1.48817014754487744,511697.78825234022365055,17635.174568,418433359.999246588003019295,17335.746838,411328777.57064553725068836,29864,267991996.8676471508404116756692698479462,273309714.3452320901855658190572173688861381504,0.324220509097
5143,btc,2023-02-02 00:00:00+00:00,983946,150681969171.599475637387303482,1.184795889398,0.853464431963,453182984739.087481763485849248,453165872083.4987139552,326449105796.707377751492843464,382498782106.23114,...,1.505769046247333252,505038.588766843750574802,21770.600903,511731142.01279010987877292,25306.969679,594855629.048353884159735864,31856,283641162.9847545459781371465523993582152,274045496.5336955395353810944076750730926618814,0.324875975436
5144,btc,2023-02-03 00:00:00+00:00,988867,150273538776.28805361363899712,1.181415746292,0.850970452062,452049702984.097911637682895936,452034246444.1374432096,325610134544.146004676584546904,382633890231.091309,...,1.86159756628409904,591390.603781686897718032,16853.572203,395145705.244925307137855328,20734.769646,486143535.38560819750909188,29804,260167412.4006683618893742396149426520538,274639194.1771995732337938940291372825022418814,0.32484282418
5145,btc,2023-02-04 00:00:00+00:00,961115,149647997797.290925613539563122,1.176437733184,0.847222179515,450328730991.77458087072933871,450312026811.61260685,324308272514.133222982192813364,382790111443.578796,...,1.49942466560198796,452943.270231851332277234,9365.480295,218735703.424879286998375474,11222.466245,262106584.054421414531276526,21809,266035849.3424773622204776160429256082152,274637599.4692072226702704255052675299466418814,0.324793322221


In [20]:
df_mcap = df_mcap[df_mcap['AdrActCnt']!=0]

In [ ]:
candles_coinbase = client.get_market_candles(
    markets="coinbase-btc-spot", # wildcards can be passed to get all asset pairs 
    start_time="2022-01-01",
    end_time="2022-02-15",
    frequency="1d"
).to_dataframe()
candles_coinbase["candle_usd_volume"] = candles_coinbase.candle_usd_volume.astype(float)
candles_coinbase["time"] = pd.to_datetime(candles_coinbase.time)
candles_coinbase.sort_values(["market","time"],inplace=True)

# Create Addt. Cols
candles_coinbase['exchange'] = candles_coinbase.market.apply(lambda x: x.split("-")[0])
candles_coinbase['exchange-base'] = candles_coinbase.market.apply(lambda x: x.split("-")[0]+"-"+x.split("-")[1])
candles_coinbase['market_type'] = candles_coinbase.market.apply(lambda x: x.split("-")[-1])
candles_coinbase['base'] = candles_coinbase.market.apply(lambda x: x.split("-")[1])
candles_coinbase['quote'] = candles_coinbase.market.apply(lambda x: x.split("-")[2])

In [9]:
# EXPORT DI TUTTI I DATI DA TUTTI GLI EXCHANGE
from typing import Optional
import logging

logger = logging.getLogger()
stream_handler = logging.StreamHandler()
level = logging.getLevelName("INFO")
stream_handler.level = level
formatter = logging.Formatter(
    datefmt="[%Y-%m-%d %H:%M:%S]", fmt="%(asctime)-15s %(levelname)s %(message)s"
)
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.level = level

# use it if you want to get specific exchanges or leave it empty if you want to get all exchanges data
# examples of echanges names:
# "binance.us",
# "binance",
# "coinbase",
# "okex",
# "kraken",
# "huobi",
# "bitmex",
# "bitfinex",
# "deribit",
# "cme",
EXCHANGES_TO_EXPORT = {}

# use it if you want to get specific markets or leave it empty if you want to get all markets
# example of market name to be used in this filter: "binance-BTCUSDT-future",
# note that if you specified exchanges filter, it will act as selecting intersection with the markets to export
# not as union.
MARKETS_TO_EXPORT = {}


# example values: "spot", "future", "option"
# you can use all 3 if you want or just a subset
MARKET_TYPES_TO_COLLECT = {
    "spot"
    
}

# leave it empty to catch all
BASE_MARKETS = {
    "btc",
}

# leave it empty to catch all
QUOTE_MARKETS = {
    "usdt",
}

# 1m, 5m, 10m, 15m, 30m, 1h, 4h, 1d
FREQUENCY = "1h"

# DST_ROOT is the path where you want the data to be saved to
# start the path with 's3://' prefix to make the script save to AWS S3, example
# or omit the s3:// prefix to save to local storage
# examples:
DST_ROOT = f"./{SOURCE}"

EXPORT_START_DATE = "2009-01-01"

# if you set EXPORT_END_DATE to None, then `today - 1 day` will be used as the end date
EXPORT_END_DATE: Optional[str] = None

# path to local file that is used to not reexport data if it was already exported
PROCESSED_DAYS_REGISTRY_FILE_PATH = "candles_processed_days_registry.txt"

client = CoinMetricsClient()

if __name__ == "__main__":
    export_start_time = datetime.datetime.now()
    try:
        my_functions.export_data(
                client,
                PROCESSED_DAYS_REGISTRY_FILE_PATH, 
                EXCHANGES_TO_EXPORT,
                MARKETS_TO_EXPORT,
                MARKET_TYPES_TO_COLLECT,
                BASE_MARKETS,
                QUOTE_MARKETS,
                EXPORT_START_DATE,
                str(datetime.datetime.now().date()), 
                DST_ROOT, 
                FREQUENCY 
        )
    finally:
        print("export took: %s", datetime.datetime.now() - export_start_time)

getting markets: %s ['bibox-btc-usdt-spot', 'binance-btc-usdt-spot', 'binance.us-btc-usdt-spot', 'bitfinex-btc-usdt-spot', 'bitstamp-btc-usdt-spot', 'bittrex-btc-usdt-spot', 'bullish-btc-usdt-spot', 'bybit-btc-usdt-spot', 'cex.io-btc-usdt-spot', 'coinbase-btc-usdt-spot', 'crypto.com-btc-usdt-spot', 'gate.io-btc-usdt-spot', 'gemini-btc-usdt-spot', 'hitbtc-btc-usdt-spot', 'huobi-btc-usdt-spot', 'kraken-btc-usdt-spot', 'kucoin-btc-usdt-spot', 'lbank-btc-usdt-spot', 'mexc-btc-usdt-spot', 'okex-btc-usdt-spot', 'poloniex-btc-usdt-spot', 'upbit-btc-usdt-spot', 'zb.com-btc-usdt-spot']
export took: %s 0:00:01.069525


In [ ]:
client.

In [10]:
import pandas as pd
from coinmetrics.constants import PagingFrom


def read_data_for_a_market(market, market_data_root, start_date, end_date, FREQUENCY, PROCESSED_DAYS_REGISTRY_FILE_PATH):                                                 
                                                                                                                                                                                                                                                                                                           
    dst_csv_file_path = "/".join((market_data_root, "candles")) + f"{FREQUENCY}.csv"                                                                                                
                                                                                                                                                           
    print("reading data from: ", dst_csv_file_path)                                                                                                               
    tmp = pd.read_csv(dst_csv_file_path)                                                                                                                         
    return tmp                                                                             

mkts = my_functions.get_markets_to_process(client,EXCHANGES_TO_EXPORT, MARKETS_TO_EXPORT, MARKET_TYPES_TO_COLLECT, BASE_MARKETS, QUOTE_MARKETS)

df = pd.DataFrame()

for mkt in mkts:
    print(mkt) 
    
    market_data_root = "/".join(                                                                                                                                    
                (                                                                                                                                                        
                    DST_ROOT.rstrip("/"),                                                                                                                                
                    mkt["market"].split("-")[0],                                                                                                                         
                    my_functions.get_instrument_root(mkt),                                                                                                                            
                ) 
    )      
    print(f'manng {my_functions.get_instrument_root(mkt)}')

    df = pd.concat([df,
                    read_data_for_a_market(
                                market_data_root,                                                                                                                                                               
                                FREQUENCY                                         
                            )
                    ],
                     ignore_index=True)       

df = df[df['volume'] == df.groupby('time')['volume'].transform('max')].sort_values(by='time').reset_index(drop=True)
df

{'market': 'bibox-btc-usdt-spot', 'min_time': '2023-07-21T14:44:05.000000000Z', 'max_time': '2023-07-22T13:58:56.846000000Z', 'exchange': 'bibox', 'type': 'spot', 'trades': {'min_time': '2023-07-21T14:44:05.000000000Z', 'max_time': '2023-07-22T13:58:56.846000000Z'}, 'base': 'btc', 'quote': 'usdt', 'symbol': 'BTC_USDT'}
manng btc_usdt_spot


TypeError: read_data_for_a_market() missing 4 required positional arguments: 'start_date', 'end_date', 'FREQUENCY', and 'PROCESSED_DAYS_REGISTRY_FILE_PATH'

In [ ]:
def get_metrics_availability():
    metrics_na = []
    metrics_a = []
    for el in client.catalog_metrics():
        metrics    = el['metric']
        frequency  = "1d"
        start_time = "2012-02-15"
        end_time   = "2022-02-16"
        asset      = ["btc","eth","bnb"]

        try:
            df_prices = client.get_asset_metrics(assets    =asset,
                                            metrics   =[metrics],
                                            frequency =frequency,
                                            start_time=start_time,
                                            end_time  =end_time).to_dataframe()
            metrics_a.append(metrics)
        except:
            print(f'Metric not found: {metrics}')
            metrics_na.append(metrics)



#client.catalog_metrics()[0]
get_metrics_availability()

In [ ]:
print(get_exchange_metrics_mapping(["coinbase", "binance"]))

In [ ]:
# Query API for bitcoin (BTC) prices, daily CM reference rates as dataframe
metrics = ["FlowInBSPUSD"]
frequency  = "1d"
start_time = "2021-01-01"
end_time   = "2022-02-16"
asset      = ["btc"]

df_prices = client.get_asset_metrics(assets    =asset,
                                     metrics   =metrics,
                                     frequency =frequency,
                                     start_time=start_time,
                                     end_time  =end_time).to_dataframe()
df_prices

In [4]:
#per ogni asset voglio prendere le metriche disponibili (che sono dati on-chain) e voglio vedere se ho a disposizione i dati storici
list_asset = ['btc','aave']
tmp = client.catalog_assets(list_asset).to_dataframe()
tmp[tmp['metrics'].notna()]


,asset,full_name,metrics,exchanges,markets,atlas
0,aave,Aave,"[{'metric': 'AdrActCnt', 'frequencies': [{'fre...","[bibox, binance, binance.us, bitfinex, bitmex,...","[bibox-aave-btc-spot, bibox-aave-eth-spot, bib...",<NA>
1,btc,Bitcoin,"[{'metric': 'AdrActCnt', 'frequencies': [{'fre...","[bibox, binance, binance.us, bitbank, bitfinex...","[bibox-aave-btc-spot, bibox-abt-btc-spot, bibo...",True


In [ ]:
tmp = client.catalog_assets('aave').to_dataframe()
for el in tmp['metrics'][0]:
    if el['metric']=='FlowInExUSD':
        print(el['metric'] )


In [ ]:
if SOURCE == 'BINANCE':
    PARAMS.update({'startTime': my_functions.datetime_to_ts(PARAMS['startTime'])})
    PARAMS.update({'endTime': my_functions.datetime_to_ts(PARAMS['endTime'])})

PARAMS  

limit=1000

In [ ]:
compare_other_symbols = True
# new_connection
client = Spot()

df = my_functions.load_data(client, PAIR, DATA_PATH, PARAMS, COLUMNS)


if compare_other_symbols:

    pairs_available = my_functions.get_all_pairs_available(client)

    # prendo le top coin e rimuovo il simbolo in cfg se c'è
    symbols, actualized_mc = my_functions.get_top_symbols_by_marketcap(50, cfg['STABLECOIN'])
    symbols = [el for el in symbols if el not in PAIR]

    for s in symbols:

        if s in pairs_available:
            
            df = pd.concat([df,my_functions.load_data(client, s, DATA_PATH, PARAMS, COLUMNS)], ignore_index=True)


In [ ]:
df.drop(columns=['OpenTimestamp', 'Ignore.'], inplace=True, errors='ignore')
df['Mkt_Cap'] = df['Close']*df['Volume']

df_mkt_cap = df.groupby(['CloseTimestamp'])['Mkt_Cap'].sum().reset_index()
df_mkt_cap.head(5)

In [ ]:
df_btc = pd.merge(df[df['Symbol']==PAIR], df_mkt_cap, on='CloseTimestamp', suffixes=[None, '_tot'])
df_btc.drop(columns=['Symbol', 'Quote asset volume','Taker buy base asset volume','Taker buy quote asset volume'], inplace=True)
df_btc['CloseTimestamp'] = df_btc['CloseTimestamp'].dt.date
df_btc.head(5)

In [ ]:
df_btc['Norm_wgt_vol'] = df['Volume']*(df_btc['Open']+df_btc['High']+df_btc['Low']+df_btc['Close'])/(4*df_btc['Number of trades'])

In [ ]:
test